# Equivariant Neural Rendering

In [ ]:
import random, os, sys
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import image as mpimg
%matplotlib inline
sns.set() # Setting seaborn as default style even if use only matplotlib
sns.set_style("whitegrid", {'axes.grid' : False})
import imageio
import torch
import torchvision
from torchvision.transforms import ToTensor


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


sys.path.append('/content/equiv-neural-rendering/')
from models.neural_renderer import *

""" Detect local path """
local_path = !pwd
local_path = local_path[0]


## 1. Introduction


The paper from Dupont et al. introduces an approach to render 2D images into implicit, equivariant 3D representations. The authors argue that the scene representations need not be explicit, as long as the transformations to it occur in an equivariant manner. Their model is trained on a dataset of rotation symmetries, learning to produce novel views from a single image of a scene.

Current approaches in scene representations present difficulties with scalability. Voxel grids, point clouds and other traditional methods have high computational and memory requirements. Reconstrucion from incomplete or noisy data is also a challenging task with these methods, often requiring 3D information during training. Generating novel views of a scene given some input views presents the same difficulties. Finally, traditional neural networks are not equivariant with respect to general transformation groups. 3D equivariance especially requires specifc techniques like steerable filters. The authors attempt to solve these problems by proposing a new method which results in more scalable, implicit representations that are also equivariant with respect to transformations. 

The difference between an explicit scene representation (mesh grid) and an implicit one can be seen in the figure below:

![Alt text](figg2.png)

  
### 1.1: Methodology

The proposed model uses a series of convolutions to map scene representations to images. Specifically, the scene representation is passed through 3D convolutions, followed by 1x1 convolutions and a set of 2D convolutions that maps them to image space. The reverse renderer is the transpose of this operation. Equivariance is enforced by applying transformations in both the scene and image spaces. Because the representation space is a deep voxel grid, the transformations in this space are defined by a 3D rotation matrix. Inverse warping with trilinear interpolation is also used in the model, to reconstruct the values after rotations.

![Alt text](fig5.png)

For training, two images of the same scene are passed through the renderer. Then, the 3D transformation grid is applied in the representation space to turn one into the other, before passing them to the inverse renderer. Finally, the output images are compared to the original inputs to obtain the loss values. Training in this manner ensures the model learns equivariant representations, as the loss evaluates both the actual rendering and the accuracy of the matching transformations from both spaces.

![Alt text](fig4.png)

Finally, the authors claim that the rendering loss used makes little change in results. They provide l1 norm, l2 norm and SSIM loss as candidates, and conduct ablation studies to determine the tradeoffs between them.

### 1.2: Datasets
The authors evaluate their model on 4 datasets, including two ShapeNet benchmarks as well as two novel datasets of the authors design. They use an image size of 128 x 128 and a representation size of 64 x 32 x 32 x 32.
The datasets are presented in table 1. 


| *Dataset*  | *Source*  |  *Sample* | *# Scenes*  |*# images per scene*| *# datapoints*|
|---|---|---|---|---||
| Chairs  | [ShapeNet](https://icml20-prod.cdn-apple.com/eqn-data/data/chairs.zip)  | ![Chair](imgs/paper/chair.png)  |  6591 | 50  | 329 550|
| Cars  |  [ShapeNet](https://icml20-prod.cdn-apple.com/eqn-data/data/cars.zip) | ![Car](imgs/paper/car.png)  |  3514 |  50 | 175 700|
| MugsHQ  |  [Apple](https://icml20-prod.cdn-apple.com/eqn-data/data/mugs.zip) | ![Mug](imgs/paper/mug.png)  |  214 | 150  | 32 100|
| 3D mountainset  |  [Apple](https://icml20-prod.cdn-apple.com/eqn-data/data/mountains.zip) | ![Mountain](imgs/paper/mountain.png)  |  559 |  50 | 27 950|

Table 1.: *Overview of datasets considered for equivariant neural rendering by Dupont et al.*

### 1.3: Experiments of paper

The proposed model is compared against three baseline models, each one making assumptions much stronger than the original study.

|   | TCO  |  DGQN | SRN  | Proposed model  |
|---|---|---|---|---|
| Requires Absolute Pose  | Yes  | Yes | Yes | No |
| Requires Pose at Inference Time  | No  | Yes | Yes | No |
| Optimization at Inference Time  | No  | No | Yes | No |

The qualitative comparisons against the baseline models in single shot novel view synthesis with the ShapeNet chairs dataset reveals that the model achieves similar to SoTA results while making far fewer assumptions than the other methods. It can produce high quality novel views by achieving the desired equivariant transformation in representation space.

![Alt text](results.png)

Results similar to the chairs were reported in the other datasets, with some variations due to the specific challenges of each one. For example, the mountains contain extremely complex geometric information, which severly limits the detail of the novel view synthesis.

![Alt text](chairs.png) 

![Alt text](cars.png) 

![Alt text](mugs.png) 

![Alt text](mountains.png)

Finally, the authors performed ablation studies to test novel view synthesis when using different loss functions. The results in each one were similar and no inherent prefered approach was suggested. In the end, they reason that choice of loss function is task specific.


### 1.4 Demo

The subsequent section presents a demonstration of the experiments conducted in the original paper.

**Configure Transformations**

In [ ]:
# ---------------------------------------------------------
# Configure transformations for demonstration
# ---------------------------------------------------------
"""
In this cell you can configure the transformations you wish 
to apply to the rendered image. You can set these to any value you like!
"""

transformations = {}

# Positive (negative) values correspond to moving camera to the right (left)
transformations['azimuth'] = 30.

# Positive (negative) values correspond to moving camera up (down)
transformations['elevation'] = 30.

# Translation values
transformations['translation'] = {'x':.2, 'y': -.2, 'z': 0.}

print(transformations)

**Define some helper functions**

In [ ]:
def tensor_to_image(img, nrow=4):
    """Helper function to plot image tensors.
    
    Args:
        img (torch.Tensor): Image or batch of images of shape 
            (batch_size, channels, height, width).
    """
    
    img_grid = torchvision.utils.make_grid(img, nrow=nrow)
    return img_grid.cpu().numpy().transpose(1, 2, 0)

def rotate_scene(model,scene, transformations):
    """Helper function to transform scene
    
    Args: 
        transformations (dict): dict comprised of keys
            azimuth: azimuth angle
            
     As a rotation matrix can feel a little abstract, we can also reason in terms of 
         camera azimuth and elevation. The initial coordinate at which the source image
         is observed is given by the following azimuth and elevation. Note that these
         are not necessary to generate novel views (as shown above), we just use them 
         for convenience to generate rotation matrices
    """
    # Set Transformations
    azimuth_shift = torch.Tensor([transformations['azimuth']]).to(device)
    elevation_shift = torch.Tensor([transformations['elevation']]).to(device)
    translations_shift = torch.Tensor([transformations['translation']['x'],
                                       transformations['translation']['y'],
                                       transformations['translation']['z']]).to(device)
    
    
    # Configure initial view
    azimuth_source = torch.Tensor([0.]).to(device)
    elevation_source = torch.Tensor([0.]).to(device)
    translations_source = torch.Tensor([0., 0., 0.]).to(device)

    # Configure output view
    azimuth_target = azimuth_source + azimuth_shift
    elevation_target = elevation_source + elevation_shift
    translations_target = translations_source + translations_shift

    # Rotate scene to match target camera angle
    transformed_scene = model.rotate_source_to_target(
        scene, 
        azimuth_source, elevation_source, translations_source,
        azimuth_target, elevation_target, translations_target
    )
    
    # Render rotated scene
    rendered = model.render(transformed_scene)
    image = tensor_to_image(rendered.detach())
    
    return image

**Loading, plotting & transforming the original image**

The model infers from a single image and renders a second image from a novel view.

In [ ]:
def visualize_model(model_path,img_path, transformations):
    # Store transformed images
    images = []
    keys   = []

    # ---------------------------------------------------------
    # Loading the original image
    # ---------------------------------------------------------

    # Load trained chairs modellocal_path
    path = local_path + model_path
    model = load_model(path).to(device)

    # You can also try loading other examples (e.g. 'chair1.png')
    path  = local_path + img_path
    original_img = plt.imread(path)

    images.append(original_img)
    keys.append('Original Image')

    # ---------------------------------------------------------
    # Render original image without transformations
    # ---------------------------------------------------------

    # Convert image to tensor and add batch dimension
    img_source = ToTensor()(original_img)
    img_source = img_source.unsqueeze(0).to(device)

    # Infer scene representation
    scene = model.inverse_render(img_source)

    # We can render the scene representation without rotating it
    rendered = model.render(scene)

    rendered_image = tensor_to_image(rendered.detach())
    images.append(rendered_image)
    keys.append('Rendered Image')
    #org = rendered.detach().clone()

    # ---------------------------------------------------------
    # Transform and render image 
    # ---------------------------------------------------------
    
    rotations = copy.deepcopy(transformations)
    
    
    rotations['translation']['x'] = 0.
    rotations['translation']['y'] = 0.
    rotations['translation']['z'] = 0.
    roto_scene = rotate_scene(model, scene, rotations)
    images.append(roto_scene)
    keys.append('Rotated Image')
    
    translations = copy.deepcopy(transformations)
    translations['azimuth'] = 0.
    translations['elevation'] = 0.

    translated_scene = rotate_scene(model, scene, translations)
    images.append(translated_scene)
    keys.append('Translated Image')
    

    roto_translated_scene = rotate_scene(model, scene, transformations)
    images.append(roto_translated_scene)
    keys.append('Roto-Translated Image')
    
    return images, keys


In [ ]:
# ---------------------------------------------------------
# Loading and plotting the original image
# ---------------------------------------------------------
# Load trained chairs modellocal_path
model_path = '/trained-models/chairs.pt'

# You can also try loading other examples (e.g. 'chair1.png')
img_path  = '/imgs/example-data/chair4.png'
demo_images, demo_keys = visualize_model(model_path,img_path, transformations)

fig, axs = plt.subplots(1, 2, figsize=(9.8, 3))
for ax, idx in zip(axs, range(2)):
    ax.imshow(demo_images[idx])
    ax.title.set_text(demo_keys[idx])
    
fig.suptitle('Demo of Original Model', size = 15)
fig.subplots_adjust(top=0.8)
    
fig, axs = plt.subplots(1, 3, figsize=(7, 3))
for ax, idx in zip(axs, range(3)):
    ax.imshow(demo_images[idx+2])
    ax.title.set_text(demo_keys[idx+2])
plt.tight_layout()

## 2. Response 

Much of the success of Deep Learning can be attributed to effective representation learning. Such representations do not need to be humanly interpretable, but can also be abstract. The original authors proposed an implicit 3D representation of the scene, instead of an explicit 3D representation such as mesh-grids or point clouds. By removing the need for an explicit 3D representation, they developed a model that requires no 3D supervision. It only requires 2D images with the corresponding rotation angle of the camera, that was used between these images. Their model can generate a novel view from a single image. The qualitative results of their model’s performance motivated us to extent their research.

In the original paper the authors used 3D rotations to generate novel views, meaning that they rotate a camera on a sphere around the scene. 3D rotations do not act transitively on 3D space. Therefore, we proposed to extend their model to roto-translations, with the intermediate proof-of-concept step of using translations only. The objective was to obtain a model that can generate a novel view for any camera position in 3D space, within a reasonable range of movement.

## 3. Novel Contribution

In this section we describe the novel contributions of our research.

- We introduce a method to generate training data for the equivariant neural rendering models (section 3.1).

- We show that the original model, which was solely pretrained on rotations, can already generate plausible translations along the axes running parallel to the image plane (i.e. orthogonal to the line of sight) through inductive bias (section 3.2).

- We introduce a model that has been trained on translations and a model that has been trained on roto-translations (section 3.3). This part constitutes the main contribution of our research.







### 3.1 Datasets

The authors present datasets consisting of rotational transformations. However, they do not provide instructions or tools for further data generation. To address this limitation we developed a new pipeline using blender for producing images of 3D-models under rotations, translations and roto-translations. Our pipeline can be used to increase the size of the training data, or to extend training data to new transformation groups.

The following section demonstrates the practical application of our pipeline for data production, by demonstrating how to use blender to generate new training data containing roto-translations.

#### 3.1.1  Demonstration: populating datasets for the ISO(3)-group using Blender 
Similar to Dupont et al., we perform experiments on the [ShapeNet Core](https://shapenet.org/download/shapenetcore)-Chairs benchmark. It is worth noting that the objects included in the ShapeNetCore dataset are already normalized and consistently aligned. However, the subsequent pipeline can be adapted to accommodate any 3D-object data that is processable by Blender. Here follows a brief demonstration of how data can be constructed using Blender 3.5.1.


In [ ]:
""" Detect local path """
local_path = !pwd
local_path = local_path[0]

**Run the subsequent cells once to install Blender with wget**

_NB! Installation assumes Linux-based OS._




In [ ]:
""" Install / Load wget """
%pip install wget
import wget

""" Install blender """
# Download blender 3.5.1
!wget https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.5/blender-3.5.1-linux-x64.tar.xz

# Unpack 
!tar -xvf blender-3.5.1-linux-x64.tar.xz
!rm {local_path}/blender-3.5.1-linux-x64.tar.xz

# Move and rename for shorter commands
!mv {local_path}/blender-3.5.1-linux-x64 {local_path}/data_prep/demo/blender


_Run render demo_

In order to build a roto-translational dataset pass both command line arguments: _--rotation --translation_ . To build a dataset composed only of rotations or translations pass only the singular argument respectively.

In [ ]:
""" Run Demo"""
!{local_path}/data_prep/demo/blender/blender -b --python data_prep/demo/render_blender.py -- --scene_name data --rotation --translation --scene_folder /data_prep/demo/data/model_1 --local_path {local_path}

_Display demonstration of roto-translation dataset_

In [ ]:
""" Display random sample outputs """
# Load 3 random images from output directory
path = local_path + "/data_prep/demo/output/rot_trans_dataset/data/"
random_file = [random.choice(os.listdir(path)) for img in range(3)]
images = [mpimg.imread(path + image) for image in random_file]

# Plot sampleset
fig, axs = plt.subplots(1, 3, figsize=(7, 3))
for ax, id in zip(axs, range(3)):
    ax.imshow(images[id])
plt.title('Demonstration of dataset production: Roto-translations of chair model\n', loc='right')
plt.show()

#### 3.1.2 Populating new datasets

We use the afformention pipeline to build 3 new datasets: 

   * _Rotations_: used to reproduce the results presented by Dupont et al.
   * _Translations_: used to train a model with higher capacity for translation invariance.
   * _Roto-translations_: used to train a roto-translational invariant model.
    
We downscale the datasets in order to reduce the computational costs of training the new models. For all three datasets we use the partitioning described in table 2. 


|   | **# Scenes**  |  **# Images per scene** | **Resolution**  | **# datapoints**  |
|---|---|---|---|---|
| Train  | 2306  |  50 | 64 x 64  |  115300 |
| Validataion  | 331  | 50  |  64 x 64 | 16550  |

Table 2: _Partition of new datasets_


| **Hyperparameter**  | **R**  |  **X** | **Y**  | **Z**  | **Resolution** |
|---|---|---|---|---|---|
|   | 1.5  | [-0.4, 0.4]  | [-0.3, 0.5]  | [-0.4, 0.4] | 64 x 64|

Table 3: _Hyperparameters used when populating the new dataset._

We construct the datasets by sampling poses from various views. In case of rotations the camera is placed on a sphere with a radius **R**. For each view, a value between 0 and $2\pi$ is uniformly sampled for the elevation and azimuth angle of the camera and rotated accordingly. In case of translations, for each view, a value is uniformly sampled from a range of **X**, **Y** and **Z** locations of the chair.

### 3.2 Translations through inductive bias

Through inductive bias, (reasonably small) translations, which are orthogonal to the line of sight, already work on the model that has only been trained on rotations. This is due to the fact that the model uses a CNN architecture, which is translationally equivariant along the image plane. Still, it seems interesting that the model does not seem to encode any information from the outside of the object to produce a good estimation.

*** More investigation necessary here (check if zero background is mapped to zero in 3D representation, check if translation introduces zero-padding in 3D presentation, check if zeros in 3D representation are mapped to zeros in image) ***

#### Translations only

In [ ]:
azimuth_shift = torch.Tensor([0.]).to(device)
elevation_shift = torch.Tensor([0.]).to(device)
translations_shift = torch.Tensor([0., 0.3, 0.3]).to(device)

azimuth_target = azimuth_source + azimuth_shift
elevation_target = elevation_source + elevation_shift
translations_target = translations_source + translations_shift

# Rotate scene to match target camera angle
rotated_scene = model.rotate_source_to_target(
    scene, 
    azimuth_source, elevation_source, translations_source,
    azimuth_target, elevation_target, translations_target
)

# Render rotated scene
rendered = model.render(rotated_scene)

plot_img_tensor(rendered.detach())

#### Roto-translations

In [ ]:
azimuth_shift = torch.Tensor([30.]).to(device)
elevation_shift = torch.Tensor([30.]).to(device)
translations_shift = torch.Tensor([0., 0.3, 0.3]).to(device)

azimuth_target = azimuth_source + azimuth_shift
elevation_target = elevation_source + elevation_shift
translations_target = translations_source + translations_shift

# Rotate scene to match target camera angle
rotated_scene = model.rotate_source_to_target(
    scene, 
    azimuth_source, elevation_source, translations_source,
    azimuth_target, elevation_target, translations_target
)

# Render rotated scene
rendered = model.render(rotated_scene)

plot_img_tensor(rendered.detach())




Nonetheless, translations along the line of sight do not work out-of-the-box and require explicit training. The reason for that is that the equivariant neural rendering model considers the depth dimension via incorporating its information into the channels of the CNN. More concrete, the model uses the following code:

```python
# Reshape 3D -> 2D
reshaped = inputs.view(batch_size, channels * depth, height, width)
```

Furthermore, due to the central positioning of the objects in the images, the model has problems rendering scenes that extent to the image boundaries. Therefore, we trained our model on translations first, before moving on to roto-translations

In [ ]:
azimuth_shift = torch.Tensor([0.]).to(device)
elevation_shift = torch.Tensor([0.]).to(device)
translations_shift = torch.Tensor([0.5, 0., 0.]).to(device)

azimuth_target = azimuth_source + azimuth_shift
elevation_target = elevation_source + elevation_shift
translations_target = translations_source + translations_shift

# Rotate scene to match target camera angle
rotated_scene = model.rotate_source_to_target(
    scene, 
    azimuth_source, elevation_source, translations_source,
    azimuth_target, elevation_target, translations_target
)

# Render rotated scene
rendered = model.render(rotated_scene)

plot_img_tensor(rendered.detach())

### 3.3 Extending the model

#### 3.3.0 Reproducing rotations with our dataset
![image](figs/results/rotations.gif)

#### 3.3.1 Translations

As shown in section 3.2, the original model, which was pretrained on rotation data only, can already produce valid shifts for short distances along the image plane but not along the line of sight. We produced a translation dataset for the model to learn how to deal with more extensive shifts of the camera, as well as learning how to deal with shifts of any size along the line of sight. Because the depth dimension is encoded within the channels of the convolution layers, it is plausible that the model can learn to make use of this information to produce a zoom-like effect, similar to which one would expect when moving the camera closer to the object.

Since we can only cover a finite range of translations, we limit the range of shift to -0.5 and 0.5 for all directions. This value was chosen because it still leaves the chair mostly in the image and hence, still contains useful information for training and inference.

*** Show the new model and present the results ***
**Demo in the making**
![image](figs/results/translations.gif)


*** Depending on how our experiments go, we might need to talk about model architecture and more inductive biases here ***

In [ ]:
### """ DEMO not working yet 
"""
# ---------------------------------------------------------
# Loading and plotting the original image
# ---------------------------------------------------------
# Load trained chairs modellocal_path
model_path = '/train_results/2023-05-14_16-54_trans_lr2e-4/best_model.pt'

# You can also try loading other examples (e.g. 'chair1.png')
img_path  = '/imgs/example-data/chair4.png'
demo_images, demo_keys = visualize_model(model_path,img_path, transformations)

fig, axs = plt.subplots(1, 2, figsize=(9.8, 3))
for ax, idx in zip(axs, range(2)):
    ax.imshow(demo_images[idx])
    ax.title.set_text(demo_keys[idx])
    
fig.suptitle('Demo of Original Model', size = 15)
fig.subplots_adjust(top=0.8)
    
fig, axs = plt.subplots(1, 3, figsize=(7, 3))
for ax, idx in zip(axs, range(3)):
    ax.imshow(demo_images[idx+2])
    ax.title.set_text(demo_keys[idx+2])
plt.tight_layout()

#### 3.3.2 Roto-Translations

The motivation for our research was to extend the equivariant neural rendering model to be able to produce novel views for any camera position in 3D space. Roto-translations fulfill this requirement. More specifically, we can cover all 3D positions by limiting the azimuth angle to a range of -180 to 180 degrees and the elevation angle to a range of -90 to 90 degrees. The latter avoids a flipping of the camera which causes problems with the compatibility of translations in Blender vs translations in the model. Nonetheless, by being able to rotate the camera along the azimuth, we can still cover all necessary views. For translations, we can obviously only cover a finite distances, therefore we chose the same range as in section 3.2 (-0.5 to 0.5 for all directions).

*** Show the new model and present the results ***

## 4. Conclusion

- Some preliminary results (working model)

## 5. Contributions 

Close the notebook with a description of the each students' contribution.